I have used CNN for predicting values mean values using npy files.
Though accuracy seems to be only ~70%. please let me know I can improve my cnn model.


## STILL WORK IN PROGRESS!!

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_json("../input/stanford-covid-vaccine/train.json",lines=True)
test = pd.read_json("../input/stanford-covid-vaccine/test.json",lines=True)
ss = pd.read_csv("../input/stanford-covid-vaccine/sample_submission.csv")

train = train.set_index('index')
test = test.set_index('index')

In [ ]:
# read npy data file
bpps_dir = '../input/stanford-covid-vaccine/bpps/'
bpps_list = os.listdir(bpps_dir)
bpps_npy = np.load(f'../input/stanford-covid-vaccine/bpps/{bpps_list[25]}')
print('Count of npy files: ', len(bpps_list))
print('Size of image: ', bpps_npy.shape)
#Size of all .npy files are not same

In [ ]:
targets = ['reactivity','deg_Mg_pH10','deg_Mg_50C','deg_pH10','deg_50C']

In [ ]:
train = train[['id']+targets ]

In [ ]:
train['reactivity'] = train['reactivity'].apply(lambda x: np.mean(x))
train['deg_Mg_pH10'] = train['deg_Mg_pH10'].apply(lambda x: np.mean(x))
train['deg_Mg_50C'] = train['deg_Mg_50C'].apply(lambda x: np.mean(x))
train['deg_pH10'] = train['deg_pH10'].apply(lambda x: np.mean(x))
train['deg_50C'] = train['deg_50C'].apply(lambda x: np.mean(x))

In [ ]:
train

In [ ]:
train_data_ids = train['id'].values

In [ ]:
train_img = []
for ID in train_data_ids:
    img_path = os.path.join(bpps_dir,ID+'.npy')
    img = np.load(img_path)
    train_img.append(img)

In [ ]:
y = train[targets].values

In [ ]:
train_img = np.array(train_img).reshape(-1, 107, 107, 1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_img, y, test_size=0.1, random_state=32)

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [ ]:
# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(107, 107,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(5, activation='linear'))

opt = Adam(lr=0.005)
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])
model.summary()

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')

callbacks = [reduce_lr]

history = model.fit(
    x=X_train,
    y=y_train,
    epochs=50,
    validation_data = (X_val,y_val),
    callbacks=callbacks
)

In [ ]:

plt.figure(figsize=(15,7))
ax1 = plt.subplot(1,2,1)
ax1.plot(history.history['loss'], color='b', label='Training Loss') 
ax1.plot(history.history['val_loss'], color='r', label = 'Validation Loss',axes=ax1)
legend = ax1.legend(loc='best', shadow=True)
ax2 = plt.subplot(1,2,2)
ax2.plot(history.history['accuracy'], color='b', label='Training Accuracy') 
ax2.plot(history.history['val_accuracy'], color='r', label = 'Validation Accuracy')
legend = ax2.legend(loc='best', shadow=True)

In [ ]:
test

In [ ]:
test_public = test[test.seq_length == 107]
test_private = test[test.seq_length == 130]

In [ ]:
test_public_ids = test_public['id'].values
test_private_ids = test_private['id'].values

In [ ]:
test_public_img = []
for ID in test_public_ids:
    img_path = os.path.join(bpps_dir,ID+'.npy')
    img = np.load(img_path)
    test_public_img.append(img)

test_private_img = []
for ID in test_private_ids:
    img_path = os.path.join(bpps_dir,ID+'.npy')
    img = np.load(img_path)
    test_private_img.append(img)
    

In [ ]:
plt.imshow(test_private_img[0])

In [ ]:
test_public_img = np.array(test_public_img).reshape(-1, 107, 107, 1)
test_private_img = np.array(test_private_img).reshape(-1, 130, 130, 1)

In [ ]:
pred_public = model.predict(test_public_img)
#pred_private = model.predict(test_private_img)

In [ ]:
len(test_private_img)

In [ ]:
pred_private.shape

In [ ]:
pred_public = np.repeat(pred_public,repeats=107,axis=0)

In [ ]:
pred_private = np.repeat(np.array([0,0,0,0,0]),repeats=130*3005,axis=0).reshape(-1,5)

In [ ]:
prediction = np.concatenate((pred_public,pred_private),axis=0)

In [ ]:
sub = pd.DataFrame(prediction)

In [ ]:
seqpos = ss.id_seqpos.values

In [ ]:
sub['id_seqpos'] = seqpos

In [ ]:
sub

In [ ]:
sub = sub.rename(columns={0: "reactivity", 1: "deg_Mg_pH10",2: "deg_Mg_50C", 3: "deg_pH10", 4: "deg_50C"})

In [ ]:
sub.to_csv("submission.csv",index=False)